In [209]:
import os
import cv2 as cv
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf


warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

TRAIN_PATH = '/Users/lydiazoghbi/Desktop/Skin Detection/Segmentation/Skin_Datasets/Dataset1_HGR/original_images/'
MASK_PATH = '/Users/lydiazoghbi/Desktop/Skin Detection/Segmentation/Skin_Datasets/Dataset1_HGR/skin_masks/'

train_id = next(os.walk(TRAIN_PATH))[2]
mask_ids = next(os.walk(MASK_PATH))[2]

train_id.sort()
train_ids = train_id[1:] #In case some weird crap .DStore file popped up
mask_ids.sort()

In [187]:
# Extract Images and Masks
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    
for n, id_ in tqdm(enumerate(mask_ids), total=len(mask_ids)):
    path = MASK_PATH + id_
    img = imread(path)
    img = np.expand_dims(resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
    Y_train[n] = img

Getting and resizing train images and masks ... 


100%|██████████| 899/899 [00:02<00:00, 448.35it/s]


In [246]:
# Extract Features and Corresponding Labels
dat = X_train[0]
dat = dat.reshape(16384,3)
hsv = cv.cvtColor(X_train[0], cv.COLOR_RGB2HSV)
hsv = hsv.reshape(16384,3)
lab = cv.cvtColor(X_train[0], cv.COLOR_RGB2Lab)
lab = lab.reshape(16384,3)
L = Y_train[0]
L = L.reshape(16384,1)
dat = np.concatenate((dat,hsv,lab,L),axis=1)

for i in range(len(train_ids)-1):
    i = i+1
    a = X_train[i]
    b = a.reshape(16384,3)
    hsv = cv.cvtColor(X_train[i], cv.COLOR_RGB2HSV)
    hsv = hsv.reshape(16384,3)
    lab = cv.cvtColor(X_train[0], cv.COLOR_RGB2Lab)
    lab = lab.reshape(16384,3)
    L = Y_train[i]
    L = L.reshape(16384,1)
    dat_temp = np.concatenate((b,hsv,lab,L),axis=1)
    dat = np.concatenate((dat,dat_temp))

In [252]:
np.random.shuffle(dat)
feat = dat[0:300000,0:9]
labels = dat[0:300000,9]

In [258]:
# Build the motherfucking network
model = Sequential()
model.add(Dense(9, input_dim=9, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(feat, labels, epochs=50, batch_size=10)

Epoch 1/50
300000/300000 [==============================] - 36s 120us/step - loss: 0.1125 - acc: 0.9621
Epoch 2/50
300000/300000 [==============================] - 37s 125us/step - loss: 0.0853 - acc: 0.9720
Epoch 3/50
300000/300000 [==============================] - 36s 121us/step - loss: 0.0822 - acc: 0.9728
Epoch 4/50
300000/300000 [==============================] - 37s 123us/step - loss: 0.0806 - acc: 0.97340s - loss: 0.0806 - acc
Epoch 5/50
300000/300000 [==============================] - 38s 128us/step - loss: 0.0796 - acc: 0.9740
Epoch 6/50
300000/300000 [==============================] - 37s 125us/step - loss: 0.0789 - acc: 0.9741
Epoch 7/50
300000/300000 [==============================] - 38s 126us/step - loss: 0.0784 - acc: 0.9742
Epoch 8/50
115030/300000 [==========>...................] - ETA: 23s - loss: 0.0796 - acc: 0.9735

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))